# Extract features from params

This notebook contains (ugly) code for parsing and structuring features from the "params" original feature.
Multiple features are contained there, and some require some heuristics / 'bussiness' logic to properly transform.
This transformation is time-consuming, may contain errors, and expanding the number of features too much would hurt explainability, and may create overfitting.

However, this exploratory notebook would be used for defining an (optional) loading_data feature, which would also include this new features.

## Load the dataset

In [38]:
import pandas as pd
df = pd.read_csv('../data/real_estate_ads_2022_10.csv')
raw_df = df.copy()
df

,id,market,created_at_first,updated_at,district_lon,district_lat,title,description,params,price,no_rooms,m,price_per_m,map_lon,map_lat
0,58668640,secondary,2019-04-09 16:12:54.000,2019-04-15 18:54:38.000,16.90502,52.41180,Mieszkanie z potencjałem. Rynek Jeżycki.,<p>Zapraszam do zakupu mieszkania zlokalizowan...,price<=>price<br>price<=>260000<br>price[curre...,260000.0,2,51.00,5098.04,16.896592,52.410150
1,58887800,secondary,2019-04-25 17:20:01.000,2019-05-27 11:36:14.000,16.87466,52.41572,Okazja Jeżyce!!!!!,<p>Polecam na sprzedaż mieszkanie kawalerka o ...,price<=>price<br>price<=>155000<br>price[curre...,155000.0,1,26.00,5961.54,16.904410,52.411919
2,55129916,secondary,2018-08-02 16:40:35.000,2021-11-28 01:56:53.000,16.90502,52.41180,"**Okazja * 60,1m2 * 3 pokoje * poddasze * Jeży...",<p> **Przestronne mieszkanie 3 pokojowe * na ...,price<=>price<br>price<=>280000<br>price[curre...,280000.0,3,60.10,4658.90,16.899310,52.413357
3,60011531,secondary,2019-12-05 12:45:49.000,2020-01-15 16:05:42.000,16.87466,52.41572,"Mieszkanie 2-pokojowe, Ogrody, Kampus Uam",<p>Serdecznie zapraszam do zapoznania się z of...,price<=>price<br>price<=>339000<br>price[curre...,339000.0,2,47.00,7212.77,16.883608,52.410662
4,60096194,secondary,2020-01-10 14:53:19.000,2020-02-13 15:25:07.000,16.87466,52.41572,Jeżyce 2 pokoje,<p>Polecam na sprzedaż mieszkanie z komórka lo...,price<=>price<br>price<=>325000<br>price[curre...,325000.0,2,44.00,7386.36,16.890796,52.412308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73640,59477668,secondary,2019-07-16 22:38:08.000,2019-10-14 23:01:42.000,16.94080,52.38356,Poznań - Rataje idealny układ pomieszczeń.,"<p>Drodzy Państwo,</p>\r\n<p>Zapraszam do zapo...",price<=>price<br>price<=>347000<br>price[curre...,347000.0,3,47.00,7382.98,16.955818,52.389898
73641,59479003,secondary,2019-07-17 11:18:30.000,2019-08-07 14:22:54.000,16.94080,52.38356,2 pokojowe z balkonem RATAJE os. Piastowskie,<p>Polecam na sprzedaż mieszkanie 2 pokojowe u...,price<=>price<br>price<=>299000<br>price[curre...,299000.0,2,43.70,6842.11,16.947615,52.392968
73642,59474988,secondary,2019-07-16 13:40:35.000,2019-07-23 00:30:42.000,16.90590,52.36596,"Inwestycyjne 2 niezależne pokoje, do wykończenia.",<p>Na sprzedaż przestronne mieszkanie położone...,price<=>price<br>price<=>270000<br>price[curre...,270000.0,2,45.32,5957.63,16.904862,52.369679
73643,59428272,secondary,2019-07-03 20:50:27.000,2022-06-17 17:14:34.000,16.94080,52.38356,Przestronna kawalerka z dużym tarasem Chartowo...,<p>Kontakt w sprawie oferty - Katarzyna Piotro...,price<=>price<br>price<=>229000<br>price[curre...,229000.0,1,36.30,6308.54,16.980725,52.389168


## Identify all given parameters

In [39]:
params = [[ll.split("<=>")[0] for ll in param.split("<br>")] for param in df['params'].tolist()]
print(f"Are all the list containing the same keys? {all([set(params[0]) == set(param) for param in params])}")

Are all the list containing the same keys? False


In [40]:
# Get the unique parameters
unique_params = {param for param_list in params for param in param_list}
unique_params

{'access_types',
 'build_year',
 'building_floors_num',
 'building_material',
 'building_ownership',
 'building_type',
 'construction_status',
 'equipment_types',
 'extras_types',
 'fence_types',
 'floor_no',
 'floors_num',
 'free_from',
 'garret_type',
 'heating',
 'heating_types',
 'is_bungalow',
 'location',
 'm',
 'market',
 'media_types',
 'price',
 'price[currency]',
 'price_per_m',
 'recreational',
 'remote_services',
 'rent',
 'rent[currency]',
 'roof_type',
 'roofing',
 'rooms_num',
 'security_types',
 'terrain_area',
 'vicinity_types',
 'windows_type'}

## Define regex for each param

In [41]:
import re
def get_params(param_list, param):
    param = re.escape(param)
    pattern = re.compile(f'{param}<=>(.*?)<br>')
    currencies = [re.findall(pattern, t) for t in df["params"].tolist()]

    return currencies
currencies = get_params(params, "price[currency]")

## Get new features from params

In [42]:
params = df["params"].tolist()
for param in unique_params:
    currencies = get_params(params, param)
    df[f"param_{param}"] = currencies

In [43]:
currencies = get_params(params, "price[currency]")
print(currencies)

[['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], ['PLN'], 

In [44]:
params

['price<=>price<br>price<=>260000<br>price[currency]<=>PLN<br>m<=>51<br>rooms_num<=>2<br>market<=>secondary<br>building_type<=>tenement<br>floor_no<=>floor_3<br>building_floors_num<=>3<br>building_material<=>brick<br>windows_type<=>plastic<br>heating<=><br>build_year<=>1920<br>construction_status<=>to_renovation<br>rent<=>price<br>rent<=>110<br>rent[currency]<=>PLN<br>building_ownership<=>full_ownership<br>free_from<=><br>media_types<=>cable-television<->internet<->phone<br>security_types<=><br>equipment_types<=><br>extras_types<=>basement<br>price_per_m<=>5098.04',
 'price<=>price<br>price<=>155000<br>price[currency]<=>PLN<br>m<=>26<br>rooms_num<=>1<br>market<=>secondary<br>building_type<=><br>floor_no<=>floor_4<br>building_floors_num<=>4<br>building_material<=><br>windows_type<=><br>heating<=><br>build_year<=><br>construction_status<=><br>rent<=>price<br>rent<=><br>rent[currency]<=>PLN<br>building_ownership<=><br>free_from<=><br>media_types<=><br>security_types<=><br>equipment_types<

In [45]:
df["param_price"]

0        [price, 260000]
1        [price, 155000]
2        [price, 280000]
3        [price, 339000]
4        [price, 325000]
              ...       
73640    [price, 347000]
73641    [price, 299000]
73642    [price, 270000]
73643    [price, 229000]
73644    [price, 399000]
Name: param_price, Length: 73645, dtype: object

## Do a custom transformation for params_price
Since the first key contains just the string "price"

In [46]:
df["param_price"] = df["param_price"].apply(lambda x: x[1] if x else None).astype("float")
print(sum(df["param_price"] != df["price"]))
df[df["param_price"]!=df["price"]][["price", "param_price"]]

2064


,price,param_price
15,421168.0,421168.50
65,348797.0,348796.80
107,270898.0,270898.50
111,374259.0,374259.42
117,390789.0,390789.04
...,...,...
73370,636134.0,636133.50
73407,312733.0,312732.60
73411,234992.0,234992.10
73445,267908.0,267907.50


We observe a mismatch between the price and param_price features, even if trying out different transformations

In [47]:
import math

for func in [math.floor, math.ceil, round]:
    dummy_df = pd.DataFrame()
    dummy_df["param_price"] = df["param_price"].apply(lambda x: func(x))

    print(sum(df["price"] != dummy_df["param_price"]))

1790
1831
928


No transformation makes them equivalent. We could use a flag in between using price or param_price in training

In [48]:
unique_params.remove("price")

## Transform the rest of the params
### Get the length of the different params

In [49]:
len_statistics = df[[f"param_{param}" for param in unique_params]].map(len).agg(['min', 'max', 'mean'])
len_statistics.T

,min,max,mean
param_market,1.0,1.0,1.000000
param_floors_num,1.0,1.0,1.000000
param_roofing,0.0,1.0,0.000136
param_terrain_area,0.0,1.0,0.000136
param_rent[currency],1.0,1.0,1.000000
param_windows_type,1.0,1.0,1.000000
param_rent,2.0,2.0,2.000000
param_fence_types,0.0,1.0,0.000136
param_is_bungalow,0.0,0.0,0.000000
param_extras_types,1.0,1.0,1.000000


### Check the params with maximum length greater than 1

In [50]:
len_statistics.T[len_statistics.T["max"] > 1]

,min,max,mean
param_rent,2.0,2.0,2.000000
param_m,4.0,5.0,4.000136


In [51]:
df["param_m"]

0                   [51, 2, 3, ]
1                   [26, 1, 4, ]
2                 [60.1, 3, 4, ]
3                   [47, 2, 4, ]
4                   [44, 2, 4, ]
                  ...           
73640     [47, 3, 4, 2019-07-31]
73641             [43.7, 2, 4, ]
73642            [45.32, 2, 4, ]
73643            [36.3, 1, 18, ]
73644    [67, 3, 15, 2019-09-01]
Name: param_m, Length: 73645, dtype: object

In [52]:
df[df["param_m"].apply(len) == 3]["param_m"]

Series([], Name: param_m, dtype: object)

In [53]:
import numpy as np
tmp_df = pd.DataFrame()

def standardize_param_m(x):
    # This includes some heuristics from analyzing the data
    if len(x) == 3:
        return [x[0], x[1], np.nan, x[2], np.nan]
    elif len(x) == 4:
        return [x[0], x[1], x[2], np.nan, x[3]]
    elif len(x) == 5:
        return x
    
tmp_df["param_m"] = df["param_m"].apply(standardize_param_m)

for i in range(5):
    tmp_df[f"param_m_{i}"] = tmp_df["param_m"].apply(lambda x: x[i])

### Check the correlation between variables

In [54]:
df_combined = pd.concat([raw_df, tmp_df], axis=1)

# Compute the correlation matrix
correlation_matrix = df_combined.corr(numeric_only=True)

correlation_matrix

,id,district_lon,district_lat,price,m,price_per_m,map_lon,map_lat
id,1.000000,-0.017327,0.037581,0.131131,-0.003512,0.162168,0.000459,0.018766
district_lon,-0.017327,1.000000,-0.197719,-0.054762,0.000068,-0.007933,0.236529,-0.048136
district_lat,0.037581,-0.197719,1.000000,0.029799,-0.001960,-0.002545,-0.047385,0.499927
price,0.131131,-0.054762,0.029799,1.000000,0.010126,0.763156,-0.006982,0.016527
m,-0.003512,0.000068,-0.001960,0.010126,1.000000,-0.007921,0.000631,-0.000456
price_per_m,0.162168,-0.007933,-0.002545,0.763156,-0.007921,1.000000,-0.002468,-0.005458
map_lon,0.000459,0.236529,-0.047385,-0.006982,0.000631,-0.002468,1.000000,0.520687
map_lat,0.018766,-0.048136,0.499927,0.016527,-0.000456,-0.005458,0.520687,1.000000


***IMPORTANT NOTE*** Cleaning / parsing this data (the "m" value in "params") requires too much effort and heuristics.

In this case, some features must be transformed to datetime, and all to numeric, to make this work smoothly. But I get some errors (not difficult to fix, but I should better expend my time on other, more important, tasks).

I get a sense of the features that it could contain, and they may already be extracted in the original dataset.

In the case of new features, since their name or description is not defined, it would introduce non-explainability to the model, and even some leaking (the data may be directly related to the price).

The decision is therefore to not use this data. 
In a practical setting, I would ask what this data contains (although with good cleaning, and checking correlations, this should work.)

## Check param_rent

In [55]:
df["param_rent"]

0           [price, 110]
1              [price, ]
2           [price, 168]
3        [price, 457.65]
4              [price, ]
              ...       
73640          [price, ]
73641    [price, 352.21]
73642       [price, 290]
73643          [price, ]
73644          [price, ]
Name: param_rent, Length: 73645, dtype: object

In [56]:
# easy to correct:
df["param_rent"] = df["param_rent"].apply(lambda x: x[1] if len(x)==2 else np.nan)

## Transform each param
Now that we have that each param has length 1, let's further transform them

In [57]:
try:
    unique_params.remove("m")
except:
    pass

for param in unique_params:
    try:
        df[f"param_{param}"] = df[f"param_{param}"].apply(lambda x: x[0] if len(x) == 1 else np.nan)
    except:
        pass

df

,id,market,created_at_first,updated_at,district_lon,district_lat,title,description,params,price,...,param_building_ownership,param_media_types,param_recreational,param_location,param_access_types,param_rooms_num,param_vicinity_types,param_roof_type,param_floor_no,param_security_types
0,58668640,secondary,2019-04-09 16:12:54.000,2019-04-15 18:54:38.000,16.90502,52.41180,Mieszkanie z potencjałem. Rynek Jeżycki.,<p>Zapraszam do zakupu mieszkania zlokalizowan...,price<=>price<br>price<=>260000<br>price[curre...,260000.0,...,full_ownership,cable-television<->internet<->phone,NaN,NaN,NaN,2,NaN,NaN,floor_3,
1,58887800,secondary,2019-04-25 17:20:01.000,2019-05-27 11:36:14.000,16.87466,52.41572,Okazja Jeżyce!!!!!,<p>Polecam na sprzedaż mieszkanie kawalerka o ...,price<=>price<br>price<=>155000<br>price[curre...,155000.0,...,,,NaN,NaN,NaN,1,NaN,NaN,floor_4,
2,55129916,secondary,2018-08-02 16:40:35.000,2021-11-28 01:56:53.000,16.90502,52.41180,"**Okazja * 60,1m2 * 3 pokoje * poddasze * Jeży...",<p> **Przestronne mieszkanie 3 pokojowe * na ...,price<=>price<br>price<=>280000<br>price[curre...,280000.0,...,full_ownership,cable-television<->internet<->phone,NaN,NaN,NaN,3,NaN,NaN,floor_4,
3,60011531,secondary,2019-12-05 12:45:49.000,2020-01-15 16:05:42.000,16.87466,52.41572,"Mieszkanie 2-pokojowe, Ogrody, Kampus Uam",<p>Serdecznie zapraszam do zapoznania się z of...,price<=>price<br>price<=>339000<br>price[curre...,339000.0,...,full_ownership,,NaN,NaN,NaN,2,NaN,NaN,floor_1,
4,60096194,secondary,2020-01-10 14:53:19.000,2020-02-13 15:25:07.000,16.87466,52.41572,Jeżyce 2 pokoje,<p>Polecam na sprzedaż mieszkanie z komórka lo...,price<=>price<br>price<=>325000<br>price[curre...,325000.0,...,,,NaN,NaN,NaN,2,NaN,NaN,floor_3,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73640,59477668,secondary,2019-07-16 22:38:08.000,2019-10-14 23:01:42.000,16.94080,52.38356,Poznań - Rataje idealny układ pomieszczeń.,"<p>Drodzy Państwo,</p>\r\n<p>Zapraszam do zapo...",price<=>price<br>price<=>347000<br>price[curre...,347000.0,...,full_ownership,cable-television<->internet<->phone,NaN,NaN,NaN,3,NaN,NaN,floor_2,entryphone<->roller_shutters
73641,59479003,secondary,2019-07-17 11:18:30.000,2019-08-07 14:22:54.000,16.94080,52.38356,2 pokojowe z balkonem RATAJE os. Piastowskie,<p>Polecam na sprzedaż mieszkanie 2 pokojowe u...,price<=>price<br>price<=>299000<br>price[curre...,299000.0,...,co_operative_ownership,0,NaN,NaN,NaN,2,NaN,NaN,floor_2,0
73642,59474988,secondary,2019-07-16 13:40:35.000,2019-07-23 00:30:42.000,16.90590,52.36596,"Inwestycyjne 2 niezależne pokoje, do wykończenia.",<p>Na sprzedaż przestronne mieszkanie położone...,price<=>price<br>price<=>270000<br>price[curre...,270000.0,...,co_operative_ownership,,NaN,NaN,NaN,2,NaN,NaN,floor_1,
73643,59428272,secondary,2019-07-03 20:50:27.000,2022-06-17 17:14:34.000,16.94080,52.38356,Przestronna kawalerka z dużym tarasem Chartowo...,<p>Kontakt w sprawie oferty - Katarzyna Piotro...,price<=>price<br>price<=>229000<br>price[curre...,229000.0,...,,,NaN,NaN,NaN,1,NaN,NaN,floor_6,entryphone


In [58]:
for param in unique_params:
    df[f"param_{param}"] = df[f"param_{param}"].apply(lambda x : x.split("<->") if isinstance(x, str) else [x])

len_statistics = df[[f"param_{param}" for param in unique_params]].map(len).agg(['min', 'max', 'mean'])
len_statistics.T

,min,max,mean
param_market,1.0,1.0,1.000000
param_floors_num,1.0,1.0,1.000000
param_roofing,1.0,1.0,1.000000
param_terrain_area,1.0,1.0,1.000000
param_rent[currency],1.0,1.0,1.000000
param_windows_type,1.0,1.0,1.000000
param_rent,1.0,1.0,1.000000
param_fence_types,1.0,3.0,1.000027
param_is_bungalow,1.0,1.0,1.000000
param_extras_types,1.0,8.0,2.321013


In [59]:
len_statistics.loc["max", "param_market"]

1.0

In [60]:
for param in unique_params:
    if len_statistics.loc["max", f"param_{param}"] == 1:
        df[f"param_{param}"] = df[f"param_{param}"].apply(lambda x: x[0])

In [61]:
df

,id,market,created_at_first,updated_at,district_lon,district_lat,title,description,params,price,...,param_building_ownership,param_media_types,param_recreational,param_location,param_access_types,param_rooms_num,param_vicinity_types,param_roof_type,param_floor_no,param_security_types
0,58668640,secondary,2019-04-09 16:12:54.000,2019-04-15 18:54:38.000,16.90502,52.41180,Mieszkanie z potencjałem. Rynek Jeżycki.,<p>Zapraszam do zakupu mieszkania zlokalizowan...,price<=>price<br>price<=>260000<br>price[curre...,260000.0,...,full_ownership,"[cable-television, internet, phone]",NaN,NaN,[nan],2,[nan],NaN,floor_3,[]
1,58887800,secondary,2019-04-25 17:20:01.000,2019-05-27 11:36:14.000,16.87466,52.41572,Okazja Jeżyce!!!!!,<p>Polecam na sprzedaż mieszkanie kawalerka o ...,price<=>price<br>price<=>155000<br>price[curre...,155000.0,...,,[],NaN,NaN,[nan],1,[nan],NaN,floor_4,[]
2,55129916,secondary,2018-08-02 16:40:35.000,2021-11-28 01:56:53.000,16.90502,52.41180,"**Okazja * 60,1m2 * 3 pokoje * poddasze * Jeży...",<p> **Przestronne mieszkanie 3 pokojowe * na ...,price<=>price<br>price<=>280000<br>price[curre...,280000.0,...,full_ownership,"[cable-television, internet, phone]",NaN,NaN,[nan],3,[nan],NaN,floor_4,[]
3,60011531,secondary,2019-12-05 12:45:49.000,2020-01-15 16:05:42.000,16.87466,52.41572,"Mieszkanie 2-pokojowe, Ogrody, Kampus Uam",<p>Serdecznie zapraszam do zapoznania się z of...,price<=>price<br>price<=>339000<br>price[curre...,339000.0,...,full_ownership,[],NaN,NaN,[nan],2,[nan],NaN,floor_1,[]
4,60096194,secondary,2020-01-10 14:53:19.000,2020-02-13 15:25:07.000,16.87466,52.41572,Jeżyce 2 pokoje,<p>Polecam na sprzedaż mieszkanie z komórka lo...,price<=>price<br>price<=>325000<br>price[curre...,325000.0,...,,[],NaN,NaN,[nan],2,[nan],NaN,floor_3,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73640,59477668,secondary,2019-07-16 22:38:08.000,2019-10-14 23:01:42.000,16.94080,52.38356,Poznań - Rataje idealny układ pomieszczeń.,"<p>Drodzy Państwo,</p>\r\n<p>Zapraszam do zapo...",price<=>price<br>price<=>347000<br>price[curre...,347000.0,...,full_ownership,"[cable-television, internet, phone]",NaN,NaN,[nan],3,[nan],NaN,floor_2,"[entryphone, roller_shutters]"
73641,59479003,secondary,2019-07-17 11:18:30.000,2019-08-07 14:22:54.000,16.94080,52.38356,2 pokojowe z balkonem RATAJE os. Piastowskie,<p>Polecam na sprzedaż mieszkanie 2 pokojowe u...,price<=>price<br>price<=>299000<br>price[curre...,299000.0,...,co_operative_ownership,[0],NaN,NaN,[nan],2,[nan],NaN,floor_2,[0]
73642,59474988,secondary,2019-07-16 13:40:35.000,2019-07-23 00:30:42.000,16.90590,52.36596,"Inwestycyjne 2 niezależne pokoje, do wykończenia.",<p>Na sprzedaż przestronne mieszkanie położone...,price<=>price<br>price<=>270000<br>price[curre...,270000.0,...,co_operative_ownership,[],NaN,NaN,[nan],2,[nan],NaN,floor_1,[]
73643,59428272,secondary,2019-07-03 20:50:27.000,2022-06-17 17:14:34.000,16.94080,52.38356,Przestronna kawalerka z dużym tarasem Chartowo...,<p>Kontakt w sprawie oferty - Katarzyna Piotro...,price<=>price<br>price<=>229000<br>price[curre...,229000.0,...,,[],NaN,NaN,[nan],1,[nan],NaN,floor_6,[entryphone]


### One-hot encode the other variables

In [62]:
dfs_to_concat = [df]

one_hot_columns = []

for param in unique_params:
    if len_statistics.loc["max", f"param_{param}"] > 1:
        df_exploded = df.explode(f"param_{param}")
        one_hot_df = pd.get_dummies(df_exploded[f"param_{param}"], prefix=f"param_{param}")
        one_hot_df.drop(columns=[f"param_{param}_0", f"param_{param}_"], inplace=True, errors="ignore")

        one_hot_columns.append(one_hot_df.columns)
        
        # Add the one-hot encoded dataframe to the list
        dfs_to_concat.append(one_hot_df)

# Concatenate all dataframes at once
df = pd.concat(dfs_to_concat, axis=0)

In [63]:
df

,id,market,created_at_first,updated_at,district_lon,district_lat,title,description,params,price,...,param_access_types_asphalt,param_access_types_hard_surfaced,param_vicinity_types_forest,param_vicinity_types_lake,param_security_types_alarm,param_security_types_anti_burglary_door,param_security_types_closed_area,param_security_types_entryphone,param_security_types_monitoring,param_security_types_roller_shutters
0,58668640.0,secondary,2019-04-09 16:12:54.000,2019-04-15 18:54:38.000,16.90502,52.41180,Mieszkanie z potencjałem. Rynek Jeżycki.,<p>Zapraszam do zakupu mieszkania zlokalizowan...,price<=>price<br>price<=>260000<br>price[curre...,260000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58887800.0,secondary,2019-04-25 17:20:01.000,2019-05-27 11:36:14.000,16.87466,52.41572,Okazja Jeżyce!!!!!,<p>Polecam na sprzedaż mieszkanie kawalerka o ...,price<=>price<br>price<=>155000<br>price[curre...,155000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55129916.0,secondary,2018-08-02 16:40:35.000,2021-11-28 01:56:53.000,16.90502,52.41180,"**Okazja * 60,1m2 * 3 pokoje * poddasze * Jeży...",<p> **Przestronne mieszkanie 3 pokojowe * na ...,price<=>price<br>price<=>280000<br>price[curre...,280000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60011531.0,secondary,2019-12-05 12:45:49.000,2020-01-15 16:05:42.000,16.87466,52.41572,"Mieszkanie 2-pokojowe, Ogrody, Kampus Uam",<p>Serdecznie zapraszam do zapoznania się z of...,price<=>price<br>price<=>339000<br>price[curre...,339000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60096194.0,secondary,2020-01-10 14:53:19.000,2020-02-13 15:25:07.000,16.87466,52.41572,Jeżyce 2 pokoje,<p>Polecam na sprzedaż mieszkanie z komórka lo...,price<=>price<br>price<=>325000<br>price[curre...,325000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,False,False,False,False,False
73642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,False,False,False,False,False
73643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,False,False,True,False,False
73644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,True,False,False,False,False


## Get the number of positive one-hots

In [64]:
one_hot_columns = [o.values.tolist() for o in one_hot_columns]

Flatten the list

In [65]:
one_hot_columns = [item for sublist in one_hot_columns for item in sublist]

In [66]:
column_sums = one_hot_df.sum()

sorted_column_sums = column_sums.sort_values(ascending=True)
sorted_column_sums

param_security_types_alarm                  1109
param_security_types_roller_shutters        2405
param_security_types_closed_area            8540
param_security_types_monitoring             9577
param_security_types_anti_burglary_door    10744
param_security_types_entryphone            27902
dtype: int64

In [67]:
# We can drop columns with less than 20 positive entries
filtered_columns = sorted_column_sums[sorted_column_sums < 20].index.tolist()
print(len(df.columns))
df.drop(columns=filtered_columns, inplace=True, errors="ignore")
print(len(df.columns))

90
90


### One-hot-encode the other features

In [68]:
unique_values = df[[f"param_{param}" for param in unique_params if len_statistics.loc["max", f"param_{param}"] == 1]].nunique()
unique_values

param_market                    2
param_floors_num               31
param_roofing                   3
param_terrain_area              9
param_rent[currency]            4
param_windows_type              4
param_rent                      9
param_is_bungalow               0
param_price[currency]           2
param_building_floors_num      29
param_garret_type               3
param_remote_services           2
param_price_per_m              10
param_heating_types             2
param_building_material        11
param_build_year              193
param_building_type            10
param_free_from              1388
param_heating                   7
param_construction_status       4
param_building_ownership        5
param_recreational              2
param_location                  2
param_rooms_num                11
param_roof_type                 3
param_floor_no                 15
dtype: int64

In [69]:
one_hot_numeric_columns = ["param_rooms_num", "param_building_floors_num", "param_floors_num", "param_free_from", "param_floor_no", "param_terrain_area", "param_rent"]

for column in [f"param_{param}" for param in unique_params if len_statistics.loc["max", f"param_{param}"] == 1]:
    if column not in one_hot_numeric_columns:
        print(column)

param_market
param_roofing
param_rent[currency]
param_windows_type
param_is_bungalow
param_price[currency]
param_garret_type
param_remote_services
param_price_per_m
param_heating_types
param_building_material
param_build_year
param_building_type
param_heating
param_construction_status
param_building_ownership
param_recreational
param_location
param_roof_type


In [70]:
df["param_rent[currency]"].value_counts()

param_rent[currency]
PLN    63176
       10467
EUR        1
USD        1
Name: count, dtype: int64

In [71]:
df["param_floors_num"].value_counts()

param_floors_num
4             22720
5             13865
               7649
3              6845
6              4550
10             3527
2              3441
8              2443
11             1893
7              1613
16             1134
1              1017
12              874
15              664
14              461
9               361
13              274
17              201
18               69
22               14
21                7
19                5
20                5
two_floors        4
one_floor         3
45                1
534810500         1
64                1
24                1
65                1
166               1
Name: count, dtype: int64

In [72]:
df["param_price_per_m"].dropna().unique()

array(['7833.33', '5177.51', '4250', '6122.81', '5403.23', '9019.03',
       '7231.99', '4800', '8375.31', '8125'], dtype=object)